In [1]:
# import required libraries
from kafka import KafkaConsumer, KafkaProducer
import avro.schema
import avro.io
import io
import hashlib, json
import threading
import time

In [2]:
def serialize(schema, obj):
    bytes_writer = io.BytesIO()
    encoder = avro.io.BinaryEncoder(bytes_writer)
    writer = avro.io.DatumWriter(schema)
    writer.write(obj, encoder)
    return bytes_writer.getvalue()

In [3]:
def deserialize(schema, raw_bytes):
    bytes_reader = io.BytesIO(raw_bytes)
    decoder = avro.io.BinaryDecoder(bytes_reader)
    reader = avro.io.DatumReader(schema)
    return reader.read(decoder)

In [4]:
schema_file = 'transaction.avsc'
txschema = avro.schema.parse(open(schema_file).read())
schema_file = 'submit.avsc'
submitschema = avro.schema.parse(open(schema_file).read())
schema_file = 'result.avsc'
resultschema = avro.schema.parse(open(schema_file).read())

In [5]:
# Connect to kafka broker running in your local host (docker). Change this to your kafka broker if needed
kafka_broker = 'lab.aimet.tech:9092'

In [6]:
producer = KafkaProducer(bootstrap_servers=[kafka_broker])

In [7]:
txconsumer = KafkaConsumer(
    'transaction',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(txschema, x))
resultconsumer = KafkaConsumer(
    'result',
     bootstrap_servers=[kafka_broker],
     enable_auto_commit=True,
     value_deserializer=lambda x: deserialize(resultschema, x))

In [8]:
def gen_signature(txid, payer, payee, amount, token):
    o = {'txid': txid, 'payer': payer, 'payee': payee, 'amount': amount, 'token': token}
    return hashlib.md5(json.dumps(o, sort_keys=True).encode('utf-8')).hexdigest()

In [9]:
def monitor_thread():
    for message in resultconsumer:
        print(message.value)

In [ ]:
monitor = threading.Thread(target=monitor_thread, daemon=True)
monitor.start()

{'timestamp': 1743176416, 'vid': 'V084917', 'txid': 'TX01649', 'checksum': '3995ff415f7466b809be10b1000f3a77', 'code': 200, 'message': 'Confirm'}
{'timestamp': 1743176425, 'vid': 'V084917', 'txid': 'TX04095', 'checksum': '9c9cf44c0e0013c9b525d2da1e42f9e6', 'code': 200, 'message': 'Confirm'}
{'timestamp': 1743176435, 'vid': 'V084917', 'txid': 'TX07261', 'checksum': 'aaa682397a31154aa2e5a3263994e8ad', 'code': 200, 'message': 'Confirm'}
{'timestamp': 1743176445, 'vid': 'V084917', 'txid': 'TX09735', 'checksum': 'd638f2e57555fb4796011b0b67b2b9c7', 'code': 200, 'message': 'Confirm'}
{'timestamp': 1743176450, 'vid': 'V084917', 'txid': 'TX08682', 'checksum': '54b43ab66c8d24c7c7573fc67502f6c9', 'code': 200, 'message': 'Confirm'}
{'timestamp': 1743176455, 'vid': 'V084917', 'txid': 'TX07825', 'checksum': '7944d438df46f815e8126505e103867d', 'code': 200, 'message': 'Confirm'}
{'timestamp': 1743176464, 'vid': 'V084917', 'txid': 'TX09959', 'checksum': '9411c7e405ebb2e2d000bff18103df55', 'code': 200, 

In [13]:
import os
from dotenv import load_dotenv

load_dotenv()
token = os.getenv('TOKEN')
vid = os.getenv('VID')

In [14]:
print('Running TX Consumer')
for message in txconsumer:
    txid = message.value['txid']
    payer = message.value['payer']
    payee = message.value['payee']
    amount = message.value['amount']

    signature = gen_signature(txid, payer, payee, amount, token)
    submit_data = {'vid': vid, 'txid': txid, 'signature': signature}
    serialized_submit_data = serialize(submitschema, submit_data)

    producer.send('submit', serialized_submit_data)
    print(message.value)
    print(f"submit: {submit_data}")

Running TX Consumer
{'txid': 'TX01649', 'payer': 'A66471', 'payee': 'A58207', 'amount': 1845}
submit: {'vid': 'V084917', 'txid': 'TX01649', 'signature': '968980d73eb9c1d11b22c9817962cde9'}
{'txid': 'TX04095', 'payer': 'A23255', 'payee': 'A37466', 'amount': 1179}
submit: {'vid': 'V084917', 'txid': 'TX04095', 'signature': 'c000cf5e3aa35d06d305a1ee4470bcdb'}
{'txid': 'TX07261', 'payer': 'A06058', 'payee': 'A25971', 'amount': 921}
submit: {'vid': 'V084917', 'txid': 'TX07261', 'signature': '2de5e0f9a3af18ab32fa6b3d857ec9e1'}
{'txid': 'TX09735', 'payer': 'A97926', 'payee': 'A73286', 'amount': 859}
submit: {'vid': 'V084917', 'txid': 'TX09735', 'signature': '24bca8d1937ab7cc2fe675c7cebc2575'}
{'txid': 'TX08682', 'payer': 'A26732', 'payee': 'A16335', 'amount': 746}
submit: {'vid': 'V084917', 'txid': 'TX08682', 'signature': 'bc14e0af3f051311e4fc177dcadb6787'}
{'txid': 'TX07825', 'payer': 'A57361', 'payee': 'A93417', 'amount': 751}
submit: {'vid': 'V084917', 'txid': 'TX07825', 'signature': '673c2

KeyboardInterrupt: 